In [1]:
import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'title_basics.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'title_akas.csv.gz',
 'Data',
 'titleR.csv.gz']

In [2]:
YEARS_TO_GET = [2010,2011,2012]

In [3]:
basics= pd.read_csv('Data/title_basics.csv.gz')

In [4]:
errors = [ ]

In [5]:
!pip install tmdbsimple

In [6]:
with open('/Users/alisonwilliams/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [7]:
tmdb.API_KEY =  login['api-key']

In [8]:
movie = tmdb.Movies()

In [9]:
info = movie.info

In [10]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    #Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #save the info releases dictionaries
    info = movie.info()
    releases = movie.releases()
##save a "certification" key in info with the certification
    info ['certification'] = c['certification']
    return info

In [11]:
errors = [ ]

In [12]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file(filename),
    Adapted from://www.geeksforgeeks.org/append-to-json-file-using-python/"""
    with open(filename,'r+') as file:
        #first we load existing file into a dict
        file_data = json.load(file)
        ##choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # sets files current position at offset.
        file.seek(0)
        #convert back to json
        json.dump(file_data, file)

In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
# Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: create it
    if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
#Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
    movie_ids = df['tconst'].copy()
# Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    final_year_df= pd.read_json(JSON_FILE)
    final_year_df.to_json(f"{FOLDER}final_tmdb_data_{YEAR}.json",
                            compression="json", index=False)

YEARS:   0%|          | 0/3 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3848 [00:00<?, ?it/s]